In [3]:
"""
Gyors duplikáció-ellenőrző script train / val / test között.

Ellenőrzések:
1) Vannak-e duplán szereplő szövegek EGY spliten belül?
2) Ugyanaz a szöveg előfordul-e több splitben is?
3) Ugyanaz a (text, label) pár előfordul-e több splitben is?

Kimenet: sima print-ek a konzolra.
"""

from pathlib import Path
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../src"))

import config
PROCESSED_DIR = Path(config.PROCESSED_DATA_DIR)



def normalize_text(s: str) -> str:
    """
    Egyszerű normalizálás:
    - strip whitespace
    - több whitespace -> egy szóköz
    - (opcionális) lower() – ha case-insensitive akarsz lenni
    """
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = " ".join(s.split())
    # ha szeretnéd, legyen case-insensitive:
    # s = s.lower()
    return s


def load_splits():
    train_path = Path(PROCESSED_DIR / "train.csv")
    val_path = Path(PROCESSED_DIR / "val.csv")
    test_path = Path(PROCESSED_DIR / "test.csv")

    print("Train path:", train_path)
    print("Val path:  ", val_path)
    print("Test path: ", test_path)

    for p in (train_path, val_path, test_path):
        if not p.exists():
            raise FileNotFoundError(f"Hiányzik a fájl: {p}")

    df_train = pd.read_csv(train_path)
    df_val = pd.read_csv(val_path)
    df_test = pd.read_csv(test_path)

    return df_train, df_val, df_test


def check_within_split(name, df):
    print(f"\n=== {name.upper()} SPLIT ===")
    print(f"Összes minta: {len(df)}")

    df = df.copy()
    df["text_norm"] = df["text"].apply(normalize_text)

    n_total = len(df)
    n_unique_text = df["text_norm"].nunique()
    n_dup_text = n_total - n_unique_text

    print(f"- Egyedi normalizált szövegek száma: {n_unique_text}")
    print(f"- Duplikált szöveg-előfordulások száma ugyanabban a splitben: {n_dup_text}")

    # (text_norm, label) párok
    n_unique_pair = df[["text_norm", "label"]].drop_duplicates().shape[0]
    n_dup_pair = n_total - n_unique_pair

    print(f"- Egyedi (text_norm, label) párok száma: {n_unique_pair}")
    print(f"- Duplikált (text_norm, label) párok száma ugyanabban a splitben: {n_dup_pair}")

    if n_dup_text > 0:
        print("\nTOP 5 duplikált szöveg a spliten belül:")
        dup_texts = (
            df.groupby("text_norm")
            .size()
            .reset_index(name="count")
            .sort_values("count", ascending=False)
        )
        for _, row in dup_texts.head(5).iterrows():
            print(f"  count={row['count']} | text_norm='{row['text_norm'][:120]}...'")


def check_cross_splits(df_train, df_val, df_test):
    # normalizált szöveg + label készítése
    for df in (df_train, df_val, df_test):
        df["text_norm"] = df["text"].apply(normalize_text)

    train_texts = set(df_train["text_norm"])
    val_texts = set(df_val["text_norm"])
    test_texts = set(df_test["text_norm"])

    print("\n=== ÁTFEDÉSEK SPLITEK KÖZÖTT (csak text_norm alapján) ===")
    inter_train_val = train_texts & val_texts
    inter_train_test = train_texts & test_texts
    inter_val_test = val_texts & test_texts

    print(f"train ∩ val  (text_norm): {len(inter_train_val)}")
    print(f"train ∩ test (text_norm): {len(inter_train_test)}")
    print(f"val   ∩ test (text_norm): {len(inter_val_test)}")

    def show_examples(name, s):
        if not s:
            return
        print(f"\nPéldák {name} átfedésre (text_norm):")
        for i, txt in enumerate(list(s)[:5]):
            print(f"  [{i}] '{txt[:120]}...'")

    show_examples("train ∩ val", inter_train_val)
    show_examples("train ∩ test", inter_train_test)
    show_examples("val ∩ test", inter_val_test)

    # (text_norm, label) párok
    def pairs(df):
        return set(zip(df["text_norm"], df["label"]))

    train_pairs = pairs(df_train)
    val_pairs = pairs(df_val)
    test_pairs = pairs(df_test)

    inter_p_train_val = train_pairs & val_pairs
    inter_p_train_test = train_pairs & test_pairs
    inter_p_val_test = val_pairs & test_pairs

    print("\n=== ÁTFEDÉSEK SPLITEK KÖZÖTT ((text_norm, label) párok) ===")
    print(f"train ∩ val  párok: {len(inter_p_train_val)}")
    print(f"train ∩ test párok: {len(inter_p_train_test)}")
    print(f"val   ∩ test párok: {len(inter_p_val_test)}")

    def show_pair_examples(name, s):
        if not s:
            return
        print(f"\nPéldák {name} átfedő (text_norm, label) párokra:")
        for i, (txt, lab) in enumerate(list(s)[:5]):
            print(f"  [{i}] label={lab} | text_norm='{txt[:120]}...'")

    show_pair_examples("train ∩ val", inter_p_train_val)
    show_pair_examples("train ∩ test", inter_p_train_test)
    show_pair_examples("val ∩ test", inter_p_val_test)


def main():
    df_train, df_val, df_test = load_splits()

    check_within_split("train", df_train)
    check_within_split("val", df_val)
    check_within_split("test", df_test)

    check_cross_splits(df_train, df_val, df_test)

    print("\n==== Duplikáció-ellenőrzés kész. ====\n")


if __name__ == "__main__":
    main()


Train path: C:\Users\nevis\Documents\Msc_2\mélytanulás\vitmma19-pw-main\vitmma19-pw\data\processed\legal_text_decoder\train.csv
Val path:   C:\Users\nevis\Documents\Msc_2\mélytanulás\vitmma19-pw-main\vitmma19-pw\data\processed\legal_text_decoder\val.csv
Test path:  C:\Users\nevis\Documents\Msc_2\mélytanulás\vitmma19-pw-main\vitmma19-pw\data\processed\legal_text_decoder\test.csv

=== TRAIN SPLIT ===
Összes minta: 2524
- Egyedi normalizált szövegek száma: 2524
- Duplikált szöveg-előfordulások száma ugyanabban a splitben: 0
- Egyedi (text_norm, label) párok száma: 2524
- Duplikált (text_norm, label) párok száma ugyanabban a splitben: 0

=== VAL SPLIT ===
Összes minta: 541
- Egyedi normalizált szövegek száma: 541
- Duplikált szöveg-előfordulások száma ugyanabban a splitben: 0
- Egyedi (text_norm, label) párok száma: 541
- Duplikált (text_norm, label) párok száma ugyanabban a splitben: 0

=== TEST SPLIT ===
Összes minta: 541
- Egyedi normalizált szövegek száma: 541
- Duplikált szöveg-előfor